<a href="https://colab.research.google.com/github/PorkPy/LSTM-Force-Predictor/blob/master/Trainer_all_smooth_and_zeroed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
% reset -f
from __future__ import print_function
from __future__ import division
import math
import torch
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
import random
import time
import torch.nn.functional as F
from scipy.stats import norm
from matplotlib.backends.backend_pdf import PdfPages
from subprocess import call
import warnings
#warnings.filterwarnings("ignore")

## Set random seed for numpy and Torch
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print('Available devices ', torch.cuda.device_count())
print('Current cuda device ', torch.cuda.current_device())

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Check CUDA is available

In [ ]:
torch.cuda.is_available()

User defined params


In [ ]:
### MODEL PARAMETERS ###

torch.manual_seed(42)
model_num  = '90'        ## model number to save new models with
params     = '81_v40' ## which model params to load...
warm_start = False       ## ...and if to load them.
model_dir  = 'model90'   ## Directory for specific model being trained. 
seq_length = 1000         ## The length of the trajectory sequence batch.
epochs     = 201         ## Number of full passes through the whole dataset.
hidden     = 60          ## Number of nodes in the LSTM layers.
lr         = 0.01       ## Learning rate.
feature_num   =  4          ## 4 features for joint data, 6 features for cartesian data.
fc         = 1           ## Number of fully connected layers. 1 or 2.
path       = f"/content/drive/My Drive/PhD/PhD/lstm/{model_dir}/" ## Save directory.

#------------------------------------------------------------------------------
## I increased the batch size and lr by 1 order.

def model_number():
    return model_num

def load_params():
    return params

def model_directory():
    return model_dir

def get_seq_length():
    return seq_length

def get_epochs():
    return epochs

def get_warm_start():
    return warm_start

def get_hidden():
    return hidden

def get_lr():
    return lr

def get_path():
    return path

def get_features():
    return feature_num

def get_fc():
    return fc

## Dictionary with which to save paramers.
param = {'Model Num':model_num, 
          'Seq Length': seq_length,
          'Epochs': epochs,
          'Warm Start': warm_start,
          'Hidden Size': hidden,
          'Learning Rate': lr,
          'Data': 'smooth 3 force data',
          'features': feature_num, 
          'Num LSTM Layers':2,
          'Num FC Layers':1}

## Create new directory in perent directory to save parameters.
try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)


## create a pandas data frame of the model parameters and save to csv.
param = pd.DataFrame(param, index=[0])
param.to_csv(path + "lstm_params.csv", index=False)


In [ ]:

def tests(model_name):
    
    seq_len = get_seq_length()
    model_name = model_name
    model_dir = model_directory()
    path = get_path()
    features_num = get_features()

    
    ## Create new directory in perent directory
    path = path + f"{model_name}/"
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

    stats_list = []
    pdf = PdfPages(path + f"testing_traj_pics_{model_name}.pdf")
    fig = plt.figure()

    for traj in range(len(test_batches)):
        Xtest, ytest, jim = get_test_batch(traj)
        model.eval()
        with torch.no_grad():
            model.reset_hidden_state()
            
            x = iter(Xtest)
            test_seq = Xtest[0].reshape(-1,seq_len,features_num)#.reshape(1,200,4) # input first sequence from trajectory/batch
            preds = [] # create a list to store predictions.
            for i in range(len(Xtest)): # for each sequence i in the trajectory,
                y_test_pred = model(test_seq).to(device)# send sequence to model,
                pred = torch.flatten(y_test_pred).cpu() # reshape the model output,
                preds.append(np.asarray(pred)) # and append to the list of predictions - preds.
                new_seq = next(x).reshape(-1,seq_len,features_num)#.reshape(1,200,4) # Change sequence to the next one in the list.
                test_seq = torch.cuda.FloatTensor(new_seq).view(1, seq_len, -1) # change sequence to a torch Tensor
                #model.reset_hidden_state()
                
        ## rescale the output predictions
        preds = target_scaler.inverse_transform(preds).reshape(-1,3)
        ## Vector summation - the vector sum of the 3 output predictions
        force_vec = np.sqrt((preds[:,0]**2)+(preds[:,1]**2)+(preds[:,2]**2))
        
        preds = force_vec ## reset name to comply with existing code.
        #display(force_vec)

        # Mean Absolute Error
        MAE_list = []
        for i,j in zip(preds, ytest):
            error = np.abs(i-j)
            MAE_list.append(error)
        MAE = float("{:.3f}".format(np.mean(MAE_list)))
        #print("MAE","{:.3f}".format(MAE),'N')

        # Coefficient of Variance
        mean = np.mean(data.iloc[:,-1]) # mean of all dependent variables.
        cov_list = []
        for i,j in zip(preds, ytest):
            sq_dev = (i-j)**2
            cov_list.append(sq_dev)    
        MSD = np.mean(cov_list) # mean square deviation
        RMSD = np.sqrt(MSD) # root mean square deviation
        cov = RMSD/mean # coefficient of variance
        RMSD = float("{:.3f}".format(RMSD))
        cov =  float("{:.3f}".format(cov))
        #print("COV:","{:.3f}".format(cov))
        
    
        my_dict = {'Trajectory':traj,
                'MAE': MAE, 
                'RMSD':RMSD,
                'cov': cov, # Used to normalise the RMSD accross all the data
        }
        stats_list.append(my_dict)

        def lighten_color(color, amount=0.5):
            import matplotlib.colors as mc
            import colorsys
            try:
                c = mc.cnames[color]
            except:
                c = color
            c = colorsys.rgb_to_hls(*mc.to_rgb(c))
            return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

        # Plot forces
        predicted_cases = preds
        true_cases = ytest
        # Add title and axis names
        plt.title(f'Force Trajectory {traj}');
        plt.xlabel('Sample num');
        plt.ylabel('Force (N)');
        plt.tight_layout();
        #plt.plot(jim,label='Sequence')
        plt.ylim(0, 70)

        plt.plot(true_cases, color=lighten_color('b', 1.8), linewidth=3.0, label='Real Force');
        
        plt.plot(predicted_cases, color=lighten_color('b', 1.2), linewidth=1.0, label='Predicted Force');
        
        plt.legend(loc=2, prop={'size': 6})

        # save the current figure
        pdf.savefig(fig);
        # destroy the current figure
        plt.clf()

    # close the object
    # fig = plt.figure()
    # plt.plot(train_hist, label="Training loss")
    # plt.plot(test_hist, label="Test loss")
    # plt.legend();
    # pdf.savefig(fig)
    # plt.clf
    pdf.close()
    stats_list = pd.DataFrame(stats_list)
    return stats_list

###############################################################################

def stats(stats_list2, model_name):
    
    
    #display(stats_list2)
    #display(stats_list2['MAE'])

    mean_list = {
                'MAE' :float("{:.3f}".format(np.mean(stats_list2['MAE']))),
                'RMSD':float("{:.3f}".format(np.mean(stats_list2['RMSD']))),
                'cov' :float("{:.3f}".format(np.mean(stats_list2['cov'])))
    }

    std_dev = {
                'MAE' :float("{:.3f}".format(np.std(stats_list2['MAE']))),
                'RMSD':float("{:.3f}".format(np.std(stats_list2['RMSD']))),
                'cov' :float("{:.3f}".format(np.std(stats_list2['cov'])))
    }

    max_list = {
                'MAE' :float(stats_list2['MAE'].max()),
                'RMSD':float(stats_list2['RMSD'].max()),
                'cov' :float(stats_list2['cov'].max())
    }

    stats_list2 = stats_list2.append(mean_list, ignore_index=True).fillna('Grand Mean')
    stats_list2 = stats_list2.append(std_dev, ignore_index=True).fillna('Standard Dev')
    stats_list2 = stats_list2.append(max_list, ignore_index=True).fillna('Max Value')

    #display(stats_list2)
    path = get_path()
    
    ## Create new directory in perent directory
    path = path + f"{model_name}/"
    model_name = model_name

    stats_list2.to_csv(path + f"lstm_model_metrics_{model_name}.csv", index=False)
    return stats_list2

###############################################################################

def gauss_plot(stats_list2, name, error_type, num):
    import matplotlib.pyplot as plt
    model_name = name
    model_dir = model_directory()
    path = get_path()
    path = path + f"{model_name}/"

    error = error_type
    pdf = PdfPages(path + f"gauss_pic_{error}.pdf")
    fig = plt.figure()
    
    # define constants
    mu = np.mean(stats_list2.iloc[:-3,num]) 
    sigma = np.sqrt(np.var(stats_list2.iloc[:-3,num]))
    x1 = np.min(stats_list2.iloc[:-3,num])
    x2 = np.max(stats_list2.iloc[:-3,num])
    # print(mu)
    # print(sigma)
    # print(x1)
    # print(x2)

    # calculate the z-transform
    z1 = ( x1 - mu ) / sigma
    z2 = ( x2 - mu ) / sigma

    x = np.arange(z1, z2, 0.001) # range of x in spec
    x_all = np.arange(-10, 10, 0.001) # entire range of x, both in and out of spec
    # mean = 0, stddev = 1, since Z-transform was calculated
    y = norm.pdf(x,0,1);
    y2 = norm.pdf(x_all,0,1);

    # build the plot
    fig, ax = plt.subplots(figsize=(9,6));
    plt.style.use('fivethirtyeight');
    ax.plot(x_all,y2);

    ax.fill_between(x,y,0, alpha=0.3, color='b');
    ax.fill_between(x_all,y2,0, alpha=0.1);
    ax.set_xlim([-4,4]);
    ax.set_xlabel('# of Standard Deviations Outside the Mean');
    ax.set_yticklabels([]);
    ax.set_title(f'{model_name} {error} Std Dev');

    plt.savefig('normal_curve.png', dpi=72, bbox_inches='tight');
    plt.grid(True);
    #plt.show()

    # save the current figure
    pdf.savefig(fig);
    # destroy the current figure
    plt.clf()

    # close the object
    pdf.close()

###############################################################################

def prob_dist(stats_list2, name, error_type, num):    
    model_name = name
    model_dir = model_directory()
    path = get_path()
    path = path + f"{model_name}/"

    error = error_type
    pdf = PdfPages(path + f"prob_dist_pic_{error}.pdf")
    fig = plt.figure()

    import seaborn as sns
    sns.distplot(stats_list2.iloc[:-3,num], color="darkslategrey");
    plt.xlabel("Force [newtons]", labelpad=14);
    plt.ylabel("Probability of Occurence", labelpad=14);
    plt.title(f"Probability Distribution of {error}", fontsize=20);
    #plt.show()
    # save the current figure
    pdf.savefig(fig);
    # destroy the current figure
    plt.clf()
    plt.close('all') ## added this due to runtime warning, more than 20 figs open
    # close the object
    pdf.close()


In [ ]:
def test_runner(name):   
    stats_df = tests(name) # Run tests on testing data and save generated plots to Google Drive
    stats(stats_df, name) # Record stats and save to Google Drive
    for i in range(1,4): # 1 to 3 = the colunms in the stats_list DataFrame
        if i ==1:
            error_type = 'MAE' # mean absolur error
        elif i == 2:
            error_type = 'RMSE' # root mean squared error
        elif i == 3:
            error_type = 'cov' # coefficient of variance

        #prob_dist(stats_df, name, error_type, i) # Gen prob_dist and save to GD
        
        #gauss_plot(stats_df, name, error_type, i) # Gen Gauss plots and save to GD
    print("Done")

In [ ]:
class ForcePredictor(nn.Module):

    def __init__(self, n_features, n_hidden, seq_len, n_layers=2, ignore_zero=True):
        super(ForcePredictor, self).__init__()

        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            print("Running on the GPU")
        else:
            device = torch.device("cpu")
            print("Running on CPU")

        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers

        self.lstm = nn.LSTM(
          input_size=n_features,
          hidden_size=n_hidden,
          num_layers=n_layers,
          dropout=0.5)
        
        fc = get_fc() ## get num of FC layers

        if fc == 1:
            self.linear1 = nn.Linear(in_features=n_hidden, out_features=3)
            
        elif fc == 2:
            self.linear1 = nn.Linear(in_features=n_hidden, out_features=60)
            self.linear2 = nn.Linear(in_features=60, out_features=3)

        elif fc == 3:
            self.linear1 = nn.Linear(in_features=60, out_features=60)
            self.linear2 = nn.Linear(in_features=n_hidden, out_features=60)
            self.linear3 = nn.Linear(in_features=60, out_features=3)
        
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device),
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device)
        )

    def forward(self, sequences):
        
        fc = get_fc() ## get num of FC layers

        lstm_out, self.hidden = self.lstm(sequences.view(len(sequences), self.seq_len, -1),self.hidden)
        last_time_step = lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]

        if fc == 1:
            y_pred = self.linear1(last_time_step)

        if fc == 2:
            y_pred = F.leaky_relu(self.linear1(last_time_step))
            y_pred = self.linear3(y_pred)

        if fc == 3:
            y_pred = F.leaky_relu(self.linear1(last_time_step))
            y_pred = F.leaky_relu(self.linear2(y_pred))
            y_pred = self.linear3(y_pred)

       

        return y_pred

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

In [ ]:
def train_model(model):
    lr = get_lr()
    loss_fn = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#0.0007 
    print("learning rate =", lr) 
    num_epochs = get_epochs() #1600 #600
    path = get_path()
    seq_length = get_seq_length()

    start_epoch = 0
    warm_start = get_warm_start()
    if warm_start == True:      
        params = load_params() # model num and version num: 4_v100.
        PATH = f"/content/drive/My Drive/PhD/PhD/lstm/model_params{params}.pt"     
        checkpoint = torch.load(PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.cuda()
        start_epoch = checkpoint['epoch']
        loss = checkpoint['loss']
    device = torch.device("cuda")
    model = model.to(device)

    train_hist = np.zeros(num_epochs + start_epoch)
    test_hist = np.zeros(num_epochs + start_epoch)

    
    losses = []
    tot_losses = []
    test_losses = []
    tot_test_losses = []
    for t in range( num_epochs):
        
        for j in range(len(batches)):
            
            ## Fetch n samples from each trajectory to train on before updatining network.
            for start_seq in range(int(1000/seq_length)): 
                start_seqx = start_seq*seq_length ## get the next sequence start position

                train_data, train_labels, test_data, test_labels = get_batches(j, start_seqx)

                if start_seq % int(1000/seq_length) == 0:
                    model.reset_hidden_state() ## reset after every trajectory
                y_pred = model(train_data)
                loss = loss_fn(y_pred.float(), train_labels)
                losses.append(loss)
                

                if test_data is not None:
                    with torch.no_grad():
                        model.reset_hidden_state() # Requiered because hidden state was influencing the prediction. 

                        y_test_pred = model(test_data)
                        test_loss = loss_fn(y_test_pred.float(), test_labels)
                    test_hist[t] = test_loss.item()
                    test_losses.append(test_loss)

                    ## The loss will look small (<1) but that's because we are not de-scaling the output. 
                    if t % 10 == 0:  
                        print(f'Epoch {t} {j} {start_seq} train loss: {loss.item()} test loss: {test_loss.item()}')
                elif t % 10 == 0:
                    print(f'Epoch {t} {j} {start_seq} train loss: {loss.item()}')

                train_hist[t] = loss.item()

                #if j % 5 == 0:
                optimizer.zero_grad()
                #loss = (sum(losses))/len(losses)
                loss.backward()
                model = model.to(device)
                optimizer.step()
                #print(model.linear4.weight.data) # Check weights are being updated.
                #losses = []

        ## Periodically save model and show training and testing loss
        if t % 10 == 0:
            print('Saving model', '\n')
            model_num = model_number()
            model_save_name = f'model_params{model_num}_v{t}.pt'
            torch.save({
                'epoch': num_epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss':loss,},
                f"/content/drive/My Drive/PhD/PhD/lstm/{model_save_name}" 
            )

            ## Show the training and testing losses during execution.
            from matplotlib.pyplot import figure
            figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
            figure(figsize=(20,4))
            plt.plot(losses, label='Training Loss');
            plt.plot(test_losses, label='Testing Loss');
            plt.legend();
            plt.tight_layout();
            plt.show()
            plt.pause(0.01)
            mean_loss = sum(losses)/len(losses)
            mean_test_loss = sum(test_losses)/len(test_losses)
            print("\n Average Loss")
            print(mean_loss.item())
            print("\n Average Test Loss")
            print(mean_test_loss.item(),'\n')
            diff = mean_test_loss - mean_loss
            print("Difference Between Training and Testing Losses")
            print(diff.item(),'\n')
            tot_losses.append(mean_loss)
            tot_test_losses.append(mean_test_loss)
        

            name = f'model{model_num}_v{t}'
            test_runner(name)
            model.train() ## just in case it was left in .eval() mode.

            model_dir = model_directory()

            pdf = PdfPages(f"/content/drive/My Drive/PhD/PhD/lstm/{model_dir}/loss.pdf")
            fig = plt.figure();

            plt.plot(tot_losses, label='Training Loss');
            plt.plot(tot_test_losses, label='Testing Loss');
            plt.xlabel("100 epochs", labelpad=14);
            plt.ylabel("Loss", labelpad=14);
            plt.title(f"Training and Testing Losses {model_dir}", fontsize=20);
            plt.grid(True);
            plt.tight_layout();
            plt.legend();

            #plt.show()
            # save the current figure
            pdf.savefig(fig);
            # destroy the current figure
            plt.clf()
            # close the object
            pdf.close()

        losses = []
        test_losses = []

    return model.eval(), train_hist, test_hist, optimizer, t, loss_fn

This section automatically selects which dataset to download based on the feature number selection made at the start. 4 features for the joint data and 6 features for the Cartesian data. 

In [ ]:
feature_num = get_features()
if feature_num == 4:
    url = 'https://raw.githubusercontent.com/PorkPy/LSTM-Force-Predictor/master/80k_data/mean_force_components_and_vectorsum_zeroed_columns.csv'
else:
    url = 'https://raw.githubusercontent.com/PorkPy/LSTM-Force-Predictor/master/80k_data/four_joints_and_three_force.csv'

data = pd.read_csv(url)
main_seq = data

In [ ]:
display(data.head(21))

In [ ]:
features = data.iloc[:,:4]
display(features)

feature_scaler = StandardScaler()
features = feature_scaler.fit_transform(features)

targets = data.iloc[:,4:7]
display(targets)

target_scaler = StandardScaler()
targets = target_scaler.fit_transform(targets)

force_vec = pd.DataFrame(data.iloc[:,-1])

features = pd.DataFrame(features)
targets = pd.DataFrame(targets)
data = pd.concat([targets, features, force_vec], axis=1)
data.columns = [['Fx', 'Fy', 'Fz', 'joint_0', 'joint_2', 'joint_4', 'joint_5', 'force mean']]
display(data)

In [ ]:
n=1000  ## num samples per trajectory/sequence.
batchesx = [data[i:i + n] for i in range(0, len(data), n)] ## a list comprehension to build the data batches.
print(len(batchesx))

random.seed(42)
random.shuffle(batchesx)


batches = batchesx[:60] ## Training batches up to the 60th sequence/trajectory.
val_batches = batchesx[60:] ## Validation batches starting from the 60th sequence.

## Append extra validation batches to even the number training and validation batches.
## This is because the training loop performs a validation test on each iteration
## and so always needs something to validate against.  
while len(val_batches) < len(batches): 
    for i in batchesx[60:]:
        val_batches.append(i)
random.shuffle(val_batches)

test_batches = batchesx[60:] ## Testing batches, same as validation batches, without the appendages.
print(len(batches), len(val_batches), len(test_batches))

In [ ]:
def get_test_batch(batch_number):
    
    seq_size = get_seq_length() ## 1000 for testing 50-100 for training
    features_num = get_features()

    X_test = []

    data = test_batches[batch_number].reset_index(drop=True)
    data= data[:seq_size]

    if features_num == 4:
        features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    else:
        features = data[['x', 'y']]#, 'z', 'Rx', 'Ry', 'Rz']]
    features = np.asarray(features)

    

    targets = data.iloc[:,:3]
    targets = np.asarray(targets)
    targets = targets.reshape(-1,3)
    target = data.iloc[:,-1]
    
    for i in range(len(features)):           
   
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, features_num))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_test.append(temp)
        y_test = target

    ## We need a scaler object to rescale the output predictions
    ## which are scaled as a result of the scaled input features.
    ## We are not actually scaling the targets here, just using it as a reference.
    

    #y_test = target ## reasign y_test now the scaler object has been created. Now, y_test is the same as it was relative to all the testing metrics.
    
    X_test = torch.cuda.FloatTensor(X_test)
    #y_test = torch.cuda.FloatTensor(y_test)
    #print(data, '\n')
    #print( X_test.shape, y_test.shape)
    jim = target
    return(X_test, y_test, jim)
  

In [ ]:
def get_batches(batch_num, start_seq):  
    
    seq_size = get_seq_length() # 1000 = full trajectories
    features_num = get_features()

    # random.seed(batch_num)
    # random.shuffle(batches) # ive turned this off to test new cleaned data

    # Randomise the fetching of new data to break the corrolation of training.
    #print(batch_num)
    #print(type(batches[batch_num])) 
    data = batches[batch_num].reset_index(drop=True)
    data= data[start_seq:seq_size+start_seq]
    ################################################

    X_train = []
    X_test = []
    
    if features_num == 4:
        features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    else:
        features = data[['x', 'y']]#, 'z', 'Rx', 'Ry', 'Rz']]
    features = np.asarray(features)


    targets = data.iloc[:,:3]
    targets = np.asarray(targets)
    targets = targets.reshape(-1,3)


    for i in range(len(features)):           
        
        np.random.seed(42)
       
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, features_num))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_train.append(temp)
        y_train = targets
    
    ###############################
    
   
    ################################
    data = val_batches[batch_num].reset_index(drop=True)
    data= data[start_seq:seq_size+start_seq]

    if features_num == 4:
        features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    else:
        features = data[['x', 'y']]#, 'z', 'Rx', 'Ry', 'Rz']]
    features = np.asarray(features)


    targets = data.iloc[:,:3]
    targets = np.asarray(targets)
    targets = targets.reshape(-1,3)
    
    for i in range(len(features)):           
   
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, features_num))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_test.append(temp)
        y_test = targets

 #############################################


###############################################

    X_train = torch.cuda.FloatTensor(X_train) # Change data to tensors
    y_train = torch.cuda.FloatTensor(y_train)
    X_test = torch.cuda.FloatTensor(X_test)
    y_test = torch.cuda.FloatTensor(y_test)
    
    #del targets, features, data, temp, an_array
    
    #print(data)
    #print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    return(X_train, y_train, X_test, y_test)

In [ ]:
# def get_batches(batch_num, start_seq):  
    
#     seq_size = get_seq_length() # 1000 = full trajectories
#     features_num = get_features()

#     # random.seed(batch_num)
#     # random.shuffle(batches) # ive turned this off to test new cleaned data

#     # Randomise the fetching of new data to break the corrolation of training.
#     #print(batch_num)
#     #print(type(batches[batch_num])) 
#     data = batches[batch_num].reset_index(drop=True)
#     data= data[start_seq:seq_size+start_seq]
#     ################################################

#     X_train = []
#     X_test = []
    
#     if features_num == 4:
#         features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
#     else:
#         features = data[['x', 'y', 'z', 'Rx', 'Ry', 'Rz']]
#     features = np.asarray(features)

#     scaler2 = StandardScaler()
#     #joint_scaler = np.asarray(features)
#     #joint_scaler = joint_scaler.reshape(-1, features_num)
#     features = scaler2.fit_transform(features) 
#     #features = scaler2.transform(features)

#     targets = data.iloc[:,:3]
#     targets = np.asarray(targets)
#     targets = targets.reshape(-1,3)


#     for i in range(len(features)):           
        
#         np.random.seed(42)
       
#         X =(features[:i+1])
#         an_array = np.array(X)
#         shape = np.shape(X)
#         temp = np.zeros((seq_size, features_num))
#         temp[(seq_size-shape[0]):,:shape[1]] = an_array
#         X_train.append(temp)
#         y_train = targets
    
#     ###############################
    
#     scaler = StandardScaler()
#     #force_scaler = np.asarray(y_train)
#     #force_scaler = force_scaler.reshape(-1,3)
#     y_train = scaler.fit_transform(y_train)    
#     #y_train = scaler.transform(y_train)

    
    
#     ################################
#     data = val_batches[batch_num].reset_index(drop=True)
#     data= data[start_seq:seq_size+start_seq]

#     if features_num == 4:
#         features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
#     else:
#         features = data[['x', 'y', 'z', 'Rx', 'Ry', 'Rz']]
#     features = np.asarray(features)

#     #scaler2 = StandardScaler()
#     #joint_scaler = np.asarray(features)
#     #joint_scaler = joint_scaler.reshape(-1, features_num)
#     #scaler2 = scaler2.fit(joint_scaler) 
#     features = scaler2.transform(features)

#     targets = data.iloc[:,:3]
#     targets = np.asarray(targets)
#     targets = targets.reshape(-1,3)
    
#     for i in range(len(features)):           
   
#         X =(features[:i+1])
#         an_array = np.array(X)
#         shape = np.shape(X)
#         temp = np.zeros((seq_size, features_num))
#         temp[(seq_size-shape[0]):,:shape[1]] = an_array
#         X_test.append(temp)
#         y_test = targets

#  #############################################

#     #scaler = StandardScaler()
#     #force_scaler = np.asarray(y_test)
#     #force_scaler = force_scaler.reshape(-1,3)
#     #scaler = scaler.fit(force_scaler)    
#     y_test = scaler.transform(y_test)


# ###############################################

#     X_train = torch.cuda.FloatTensor(X_train) # Change data to tensors
#     y_train = torch.cuda.FloatTensor(y_train)
#     X_test = torch.cuda.FloatTensor(X_test)
#     y_test = torch.cuda.FloatTensor(y_test)
    
#     #del targets, features, data, temp, an_array
    
#     #print(data)
#     #print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
#     return(X_train, y_train, X_test, y_test)

In [ ]:
%%time

tot_losses = []
tot_test_losses = []

seq_length = get_seq_length() # when using zero padding, this seq_length is a bit redundent but still has to match the zero's size.

model = ForcePredictor(
      n_features=get_features(), 
      n_hidden= get_hidden(), #32, #64
      seq_len=seq_length, 
      n_layers=2
    )

model, train_hist, test_hist, optimizer, epochs, loss = train_model(model)



In [ ]:
print("Saving model")
model_num = model_number()
model_save_name = f'model_params{model_num}_vlast.pt'
torch.save({
   # 'epoch': epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss':loss,},
    f"/content/drive/My Drive/PhD/PhD/lstm/{model_save_name}" 
)

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
figure(figsize=(20,4))
plt.plot(train_hist[1000:], label="Training loss")

#plt.plot(test_hist, label="Test loss")
#plt.ylim((0, 5))
plt.legend();

RL Controller Predictor